In [127]:
import pandas
from os.path import isfile, join
import os 
root_path = os.getcwd()

def limpaDf(df):
        new = df[ ["Ação Orçamentária", "Município", "Valor Transferido"]].copy() 
        return new

def limpaDfIDEB(df):
        return df[ ["Município", 2017]]
        
        

In [128]:
path = "./dados/dados brutos/Transferencias 2016"
directory = os.listdir(path)
all_data_frames = []

for file in directory:
    file_path = os.path.join(path, file)
    df = pandas.read_csv( file_path ,  delimiter=";", encoding="utf-8-sig")
    df = limpaDf(df)
    all_data_frames.append(df)

acoes2016 = pandas.concat(all_data_frames)
acoes2016.to_csv(r"./dados/dados limpos/acoes2016.csv")

In [129]:
collection_data = []
columns_collection = ["Município", "Total Investido", "IDEB - 4 e 5", "IDEB - 8 e 9"]

df_ideb_5_6 = pandas.read_excel("./dados/dados brutos/IDEB Paraíba.xlsx", sheet_name="4 serie5 ano",  delimiter=";", encoding="utf-8-sig")
df_ideb_8_9 = pandas.read_excel("./dados/dados brutos/IDEB Paraíba.xlsx", sheet_name="8 Serie 9 ano",  delimiter=";", encoding="utf-8-sig")

df_ideb_5_6 = limpaDfIDEB(df_ideb_5_6)
df_ideb_8_9 = limpaDfIDEB(df_ideb_8_9)


In [130]:
import pandasql as ps

file_path =  "./dados/dados limpos/acoes2016.csv"
acoes = pandas.read_csv( file_path ,  delimiter=",", encoding="utf-8-sig")

municipios = acoes[["Município"]].copy()

municipios = municipios.drop_duplicates(subset="Município")

municipios_list = list(municipios["Município"])

for municipio in municipios_list :
    df_municipio = ps.sqldf('select * from acoes where Município = "' + municipio + '"', locals())
    ideb_municipio_4_5 = ps.sqldf('select \"2017\" from df_ideb_5_6 where '\
                              'Município like "' + municipio.capitalize() + '"', locals())
    ideb_municipio_8_9 = ps.sqldf('select \"2017\" from df_ideb_8_9 where '\
                              'Município like "' + municipio.capitalize() + '"', locals())
    total = 0
    
    for row in df_municipio["Valor Transferido"]:
        total += float("".join([i for i in row.replace("R$", "") if i != "."]).replace(",", "."))
    
    collection_data.append([
        municipio,
        total,
        "" if len(ideb_municipio_4_5["2017"]) == 0 else ideb_municipio_4_5["2017"][0],
        "" if len(ideb_municipio_8_9["2017"]) == 0 else ideb_municipio_8_9["2017"][0],
    ])
    
    df_municipio.to_csv(r"./dados/dados limpos/acoes-municipio/" + municipio + ".csv")

pandas.DataFrame(
    collection_data, columns=columns_collection
).to_csv(r"./dados/dados limpos/ideb-municipio-por-investimento.csv")